In [ ]:
import json
import requests
import tempfile
import geopandas as gpd
import pandas as pd
import leafmap.foliumap as leafmap
from ipyleaflet import (
    Map,
    SearchControl,
    LayersControl,
    TileLayer,
    WidgetControl,
    GeoJSON,
    Marker,
)
from ipywidgets import HTML

In [ ]:
search_map = Map(center=[35.96, -83.92], zoom=10, scroll_wheel_zoom=True)
search_control = SearchControl(
    position="topleft",
    url="https://nominatim.openstreetmap.org/search?format=json&q={s}",
    zoom=12,
    property_name="display_name",
    marker=Marker(),
)
search_map.add_control(search_control)
search_map.add_control(LayersControl())
search_map

In [ ]:
esa_map = leafmap.Map()
esa_map.add_wms_layer(
    url="https://services.terrascope.be/wms/v2?",
    layers="WORLDCOVER_2021_MAP",
    name="ESA WorldCover 2021",
    attribution="ESA",
    transparent=True,
)
esa_legend = {
    "Tree Cover": "#006400",
    "Shrubland": "#ffbb22",
    "Grassland": "#ffff4c",
    "Cropland": "#f096ff",
    "Built-up": "#fa0000",
    "Bare / sparse vegetation": "#b4b4b4",
    "Snow and ice": "#f0f0f0",
    "Permanent water bodies": "#0064c8",
    "Herbaceous wetland": "#0096a0",
    "Mangroves": "#00cf75",
    "Moss and lichen": "#fae6a0",
}
esa_map.add_legend(title="ESA WorldCover", legend_dict=esa_legend)
esa_map

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load building polygons
gdf_buildings = gpd.read_file(
    r"C:\Users\Alex\Documents\GeoMorphProject1\data\KnoxCounty_Buildings.geojson"
)

# Compute centroids for each building
gdf_centroids = gdf_buildings.copy()
gdf_centroids["geometry"] = gdf_centroids.centroid

# Plot the centroids to simulate a cluster map
fig, ax = plt.subplots(figsize=(12, 12))
gdf_centroids.plot(ax=ax, color="yellow", edgecolor="red", markersize=5)
ax.set_title("Simulated Building Centroid Cluster Map (Knox County)")
ax.axis("off")
plt.show()

In [ ]:
# Initialize map only once
vector_map = leafmap.Map(center=[35.96, -83.92], zoom=10)

# 1. Add building polygons (outline only)
vector_map.add_geojson(
    r"C:\Users\Alex\Documents\GeoMorphProject1\data\KnoxCounty_Buildings.geojson",
    layer_name="Buildings",
    style={"color": "red", "fillOpacity": 0},
)

# 2. Load and add roads from shapefile
roads_shp_path = (
    r"C:\Users\Alex\Documents\GeoMorphProject1\data\tl_2023_47093_roads.shp"
)
gdf_roads = gpd.read_file(roads_shp_path)
roads_geojson = gdf_roads.__geo_interface__

vector_map.add_geojson(
    roads_geojson, layer_name="Roads", style={"color": "red", "weight": 1.5}
)

# 3. Choropleth from US counties with simulated census-style data
geojson_url = "https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson"
gdf_counties = gpd.read_file(geojson_url)

# Simulated attribute data
data = {
    "FIPS": ["01001", "01003", "01005", "01007"],
    "UnemploymentRate": [5.6, 3.8, 6.1, 4.5],
}
df_attributes = pd.DataFrame(data)

# Merge on FIPS
gdf_counties["id"] = gdf_counties["id"].astype(str)
df_attributes["FIPS"] = df_attributes["FIPS"].astype(str)
gdf_merged = gdf_counties.merge(df_attributes, left_on="id", right_on="FIPS")

# Add choropleth
vector_map.add_data(
    gdf_merged,
    column="UnemploymentRate",
    cmap="YlOrRd",
    legend_title="Unemployment Rate (%)",
)

# Display everything
vector_map

In [ ]:
import leafmap

split_map = leafmap.SplitMap(center=[32.1, 20.1], zoom=10)
split_map.add_basemap("Esri.WorldImagery")

split_map.add_left_layer(
    url="https://github.com/opengeos/datasets/releases/download/v0.1/libya_flood_before.tif",
    layer_name="Before Flood",
)

split_map.add_right_layer(
    url="https://github.com/opengeos/datasets/releases/download/v0.1/libya_flood_after.tif",
    layer_name="After Flood",
)

split_map